In [78]:
import os
import sys
sys.path.append('../')

from util import load_pkl

In [ ]:
#INGRESS_HTTP_PORT="8180"
INGRESS_GPRC_PORT="30193"
#INGRESS_GPRC_PORT="8181"
INGRESS_HOST="172.21.0.3"
#INGRESS_HOST="127.0.0.1"
#INGRESS_HOST="knative-local-gateway.istio-system.svc.cluster.local"
SERVICE_HOSTNAME="dynamic-model-grpcv2.default.example.com"

In [98]:
period = 128
data_dir = f'../../processed_data/processed_data_{period}'

In [99]:
df_train_X_all = load_pkl(f"{data_dir}/df_train_X_all.pkl")
df_train_y_all = load_pkl(f"{data_dir}/df_train_y_all.pkl")
df_test_X_all = load_pkl(f"{data_dir}/df_test_X_all.pkl")
df_test_y_all = load_pkl(f"{data_dir}/df_test_y_all.pkl")

In [100]:
with open(f'{data_dir}/valid_tickers.txt', 'r') as f:
    valid_tickers = f.readlines()

In [101]:
import numpy as np

feature_file_path = f'{data_dir}/sorted_features.txt'
with open(feature_file_path, 'r') as file:
  sorted_features = np.array(file.read().split('\n'))
  sorted_features = sorted_features[sorted_features != '']


In [102]:
for i in range(len(valid_tickers)):
  df_train_X_all[i] = df_train_X_all[i][sorted_features]
  df_test_X_all[i] = df_test_X_all[i][sorted_features]

In [103]:
df_test_X = df_test_X_all[0]
stock_id = valid_tickers[0].strip()
print(stock_id)
model_name = f"{stock_id}_rf"

ADS.DE


In [105]:
import numpy as np
from tritonclient.grpc import InferenceServerClient, InferInput, InferRequestedOutput


def send_predict_request_grpc(
    url: str,
    entrypoint_model_name: str,   # e.g. "dynamic-sklearn"
    sub_model_name:  str,         # e.g. "ADS.DE_rf"

    features: np.ndarray,          # 1D or 2D numpy array
    dtype: np.dtype = np.float32,
):
    """
    Send a gRPC (V2) request to a KServe model server.

    Args:
      url:           host:port of the KServe gRPC endpoint, e.g. "localhost:8182"
      model_name:    the name you passed to DynamicSklearnModel(name=…)
      features:      either a single 1D list [f1,f2,…] or a batch [[…],[…],…]
      dtype:         numpy dtype for features (commonly float32)

    Returns:
      np.ndarray of predictions
    """

    # 1) normalize to a 2D numpy array
    arr = np.array(features, dtype=dtype)
    if arr.ndim == 1:
        arr = arr.reshape(1, -1)

    # 2) create the Triton client and inputs/outputs
    client = InferenceServerClient(url=url, verbose=True, channel_args=[('grpc.ssl_target_name_override', SERVICE_HOSTNAME)])

    # *** METHOD A:  dtype=object containing bytes ***
    model_bytes_A = np.array([sub_model_name.encode("utf-8")], dtype=object)
    print("DEBUG (A) array =", model_bytes_A, " dtype=", model_bytes_A.dtype) # <-- ADDED
    mni_A = InferInput("model_name", (1,), "BYTES")
    mni_A.set_data_from_numpy(model_bytes_A)


    # --- 3) Build the "features" FP32 tensor input
    feat = InferInput(
        "features",              # must match your Model.predict() second input name
        features.shape,
        "FP32"
    )
    feat.set_data_from_numpy(features.astype(np.float32))

    # --- 4) Declare the output
    out = InferRequestedOutput("outputs")  # or "predictions" if you named it so


    # 3) actually call the model
    response = client.infer(
        model_name=entrypoint_model_name,
        inputs=[mni_A, feat],
        outputs=[out],
        timeout=int(10.0),             # seconds to wait
    )
    print(response.get_response())
    out_name = response.get_response().outputs[0].name
    result   = response.as_numpy(out_name)
    return result

grpc_url = f"{INGRESS_HOST}:{INGRESS_GPRC_PORT}"  # gRPC endpoint

features = df_test_X.iloc[0].values.tolist() # First row
resp = send_predict_request_grpc(grpc_url, 'dynamic-model-name', model_name, np.array(features))
print("Prediction Result:", resp)

batch_size = 30 # Number of ROWS to send as batch
features_batch = df_test_X.iloc[0:batch_size].values.tolist() # First batch_size rows

resp = send_predict_request_grpc(grpc_url, 'dynamic-model-name', model_name, np.array(features_batch))
print("Prediction Result:", resp)

DEBUG (A) array = [b'ADS.DE_rf']  dtype= object
infer, metadata dict_items([])
model_name: "dynamic-model-name"
parameters {
  key: "timeout"
  value {
    int64_param: 10
  }
}
inputs {
  name: "model_name"
  datatype: "BYTES"
  shape: 1
}
inputs {
  name: "features"
  datatype: "FP32"
  shape: 28
}
outputs {
  name: "outputs"
}
raw_input_contents: "\t\000\000\000ADS.DE_rf"
raw_input_contents: "\327\2430?\250T\273>ff\206?e\353\216\274\037\205\353?\000\000\000\000\022\377\371=\t\216\352;\301\356\220=\036\t9=\232\231\231\277\n\327#<\347\344\262\275\000\000\000\277\345\255\251<\000\000\200>(6&\276\254\363B:\214\224\274:\315\314\314=/\243\221\275\214\320Z\275\315\314L\276\337\266\231<\000\000\000\000\310\361G\273\341z\224>A\331P;"

model_name: "dynamic-model-name"
id: "956f473b-d215-463d-b4da-6d761007b8c9"
outputs {
  name: "output-0"
  datatype: "FP64"
  shape: 1
  parameters {
    key: "binary_data_size"
    value {
      int64_param: 8
    }
  }
}
raw_output_contents: "\227\312\2305\30

In [50]:
batch_size = 30
total_requests = 10000  # Total number of requests you want to send
concurrent_workers = 50  # Number of threads to run concurrently


In [51]:
import requests
import json
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

def load_test(server_url, model_name, df, batch_size, total_requests, concurrent_workers):
    start_time = time.time()
    results = []
    with ThreadPoolExecutor(max_workers=concurrent_workers) as executor:
        futures = []
        for i in range(total_requests):
            batch = df.iloc[i*batch_size:(i+1)*batch_size].values.tolist()
            futures.append(executor.submit(send_predict_request_http, server_url, model_name, batch))

        for idx, future in enumerate(as_completed(futures), 1):
            result = future.result()
            results.append(result)
            if idx % 100 == 0:
                print(f"{idx} / {total_requests} requests completed")

    end_time = time.time()
    duration = end_time - start_time
    print(f"Completed {total_requests} requests in {duration:.2f} seconds")
    print(f"Requests per second: {total_requests / duration:.2f}")
    return results

In [52]:
load_test(server_url, model_name, df_test_X, batch_size, total_requests, concurrent_workers)

Prediction Result: {
  "predictions": [
    0.1041377969657475,
    0.10416572140848658,
    0.10405841038572897,
    0.10412309750137405,
    0.10407388978990052,
    0.10405349350259109,
    0.10402427052343133,
    0.10402882819272352,
    0.1040260333972378,
    0.10397991937066303,
    0.10398096697448367,
    0.10396341757426208,
    0.10393646730119266,
    0.10391329197980036,
    0.1039228660758785,
    0.10428646643411871,
    0.10432832939239872,
    0.10433436866282007,
    0.10433302855975594,
    0.1043344480148969,
    0.10432824231615369,
    0.10432179461492831,
    0.10436286574526181,
    0.10442563496344129,
    0.10440407848466597,
    0.10439489161520829,
    0.10440499650472188,
    0.10440931756990086,
    0.10440466377418664,
    0.10439009912082704
  ]
}
Prediction Result: {
  "predictions": [
    0.1044115673775878,
    0.10441286162392759,
    0.10440632613114041,
    0.10441255043522855,
    0.10440507279538322,
    0.10440759279277323,
    0.10440994581456

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,